In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [ ]:
ratings = pd.read_csv('/Users/admin/Desktop/anime-recommendations-database/rating.csv')
anime = pd.read_csv('/Users/admin/Desktop/anime-recommendations-database/anime.csv')

In [ ]:
ratings.head()

In [ ]:
anime.head()

In [ ]:
#'members'でソート
anime.sort_values('members', ascending=False).head()

In [ ]:
round(anime.describe(), 2)

In [ ]:
round(ratings.describe(), 2)

In [ ]:
ratings['rating'].hist(bins=11, figsize=(5, 5))

In [ ]:
anime['members'].hist(bins=20, figsize=(10, 5))

In [ ]:
#members 10000以上のものだけを選択
popu_anime = anime[anime['members'] > 10000]
round(popu_anime.describe(), 2)

In [ ]:
#欠損データを確認
popu_anime.isnull().sum()

In [ ]:
#欠損データ削除
popu_anime = popu_anime.dropna()

In [ ]:
#評価ついてるものだけを選択
modi_ratings = ratings[ratings.rating >= 0]

In [ ]:
round(modi_ratings.describe(), 2)

In [ ]:
modi_ratings.isnull().sum()

In [ ]:
#animeとratingをマージ
mergeddf = modi_ratings.merge(popu_anime, left_on='anime_id', right_on='anime_id', suffixes=['_user', ''])

mergeddf.head()

In [ ]:
round(mergeddf.describe(), 2)

In [ ]:
mergeddf = mergeddf[['user_id', 'name', 'rating_user']]
mergeddf = mergeddf.drop_duplicates(['user_id', 'name' ])
mergeddf.head()

In [ ]:
#データフレームをピボット
anime_pivot = mergeddf.pivot(index='name', columns='user_id', values='rating_user').fillna(0)
anime_pivot_sparse = csr_matrix(anime_pivot.values)

anime_pivot.head()

In [ ]:
#Learning model
knn = NearestNeighbors(n_neighbors=9, algorithm='brute', metric='cosine')
model_knn = knn.fit(anime_pivot_sparse)

In [ ]:
def searchanime(string):
    print(anime_pivot[anime_pivot.index.str.contains(string)].index[0:])

searchanime('Hajime')

In [ ]:
# 「はじめの一歩」に対してのオススメのアニメ10個
Anime = 'Hajime no Ippo'
distance, indice = model_knn.kneighbors(anime_pivot.iloc[anime_pivot.index== Anime].values.reshape(1,-1), n_neighbors=11)
for i in range(0, len(distance.flatten())):
    if  i == 0:
        print('Recommendations if you like the anime {0}:\n'.format(anime_pivot[anime_pivot.index== Anime].index[0]))
    else:
        print('{0}: {1} with distance: {2}'.format(i,anime_pivot.index[indice.flatten()[i]],distance.flatten()[i]))

print(distance.flatten())
print(indice.flatten())

In [ ]:
# 「君の名は」を見たことがあるあなたにオススメのアニメは・・・
Anime = 'Kimi no Na wa.'
distance, indice = model_knn.kneighbors(anime_pivot.iloc[anime_pivot.index== Anime].values.reshape(1,-1),n_neighbors=11)
for i in range(0, len(distance.flatten())):
    if  i == 0:
        print('Recommendations if you like the anime {0}:\n'.format(anime_pivot[anime_pivot.index== Anime].index[0]))
    else:
        print('{0}: {1} with distance: {2}'.format(i,anime_pivot.index[indice.flatten()[i]],distance.flatten()[i]))